# Modelagem de Dados — Previsão de Diabetes (Pima Indians Diabetes Dataset)

Este notebook contém o pipeline completo de modelagem para o Projeto Integrador:
- Carregamento do dataset
- Pré-processamento e limpeza
- Análise exploratória básica
- Treinamento e avaliação de modelos (Random Forest, Regressão Logística)
- Visualizações (matriz de confusão, curva ROC, importância das variáveis)
- Salvamento do modelo

> Observação: o dataset pode ser baixado diretamente do link público ou usado um arquivo local chamado `pima-indians-diabetes.data.csv`.


In [ ]:
# 1) Importar bibliotecas e carregar o dataset
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import pickle

# Caminhos de arquivo
csv_path_local = '/mnt/data/pima-indians-diabetes.data.csv'
csv_path_save = '/mnt/data/pima-indians-diabetes.csv'

# Colunas esperadas (o arquivo original não tem header)
cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']

# Tentar carregar: primeiro local, senão tentar ler a URL (se disponível)
if os.path.exists(csv_path_local):
    df = pd.read_csv(csv_path_local, header=None, names=cols)
elif os.path.exists(csv_path_save):
    df = pd.read_csv(csv_path_save)
else:
    try:
        url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
        df = pd.read_csv(url, header=None, names=cols)
        df.to_csv(csv_path_save, index=False)
        print('Dataset baixado e salvo em', csv_path_save)
    except Exception as e:
        raise FileNotFoundError('Nenhum dataset local encontrado e download falhou. Faça upload do CSV para /mnt/data/pima-indians-diabetes.data.csv') from e

print('Formato do dataset:', df.shape)
df.head()


In [ ]:
# 2) Pré-processamento e limpeza
df_orig = df.copy()

# Algumas colunas usam 0 como valor ausente (Glucose, BloodPressure, SkinThickness, Insulin, BMI)
cols_zero_na = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

# Substituir zeros por NaN nessas colunas
df[cols_zero_na] = df[cols_zero_na].replace(0, np.nan)

# Estatísticas antes do tratamento
missing_before = df.isna().sum()

# Imputação simples com mediana para manter robustez
imputer = SimpleImputer(strategy='median')
df[cols_zero_na] = imputer.fit_transform(df[cols_zero_na])

# Conferir depois
missing_after = df.isna().sum()

print('Valores ausentes antes:\n', missing_before)
print('Valores ausentes depois:\n', missing_after)


In [ ]:
# 3) Análise Exploratória básica
# Estatísticas descritivas
desc = df.describe()

# Contagem por classe
class_counts = df['Outcome'].value_counts()

# Correlação com target
corr_target = df.corr()['Outcome'].sort_values(ascending=False)

print('Descrição:
', desc)
print('\nContagem por classe:\n', class_counts)
print('\nCorrelação com Outcome:\n', corr_target)


In [ ]:
# 4) Dividir em treino e teste e padronizar
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Padronizar features numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Shapes:', X_train.shape, X_test.shape)


In [ ]:
# 5) Treinar modelos: Random Forest (base) e Regressão Logística
# Random Forest inicial
rf = RandomForestClassifier(random_state=42, n_estimators=200)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
acc_rf = accuracy_score(y_test, y_pred_rf)

# Regressão logística para comparação
log = LogisticRegression(max_iter=1000, random_state=42)
log.fit(X_train_scaled, y_train)
y_pred_log = log.predict(X_test_scaled)
acc_log = accuracy_score(y_test, y_pred_log)

print('Acurácia RF:', acc_rf)
print('Acurácia LogReg:', acc_log)


In [ ]:
# 6) Avaliação detalhada do Random Forest (modelo escolhido)
from sklearn.metrics import classification_report
print('Classification Report (Random Forest):\n', classification_report(y_test, y_pred_rf))
cm = confusion_matrix(y_test, y_pred_rf)
print('\nMatriz de Confusão:\n', cm)

# Curva ROC e AUC
y_proba_rf = rf.predict_proba(X_test_scaled)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_proba_rf)
roc_auc = auc(fpr, tpr)
print('\nAUC ROC:', roc_auc)

# Plotar matriz de confusão (matplotlib, sem paleta de cores explícita)
plt.figure(figsize=(4,4))
plt.imshow(cm, interpolation='nearest', aspect='auto')
plt.title('Matriz de Confusão')
plt.xlabel('Previsto')
plt.ylabel('Real')
for (i,j),val in np.ndenumerate(cm):
    plt.text(j, i, val, ha='center', va='center')
plt.colorbar()
plt.show()

# Plot ROC
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1], linestyle='--')
plt.title(f'Curva ROC (AUC = {roc_auc:.3f})')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()


In [ ]:
# 7) Importância das variáveis (Random Forest)
importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
}).sort_values(by='importance', ascending=False)

print(importances)
# Plot simples
plt.figure(figsize=(6,4))
plt.barh(importances['feature'], importances['importance'])
plt.title('Importância das variáveis (Random Forest)')
plt.xlabel('Importância')
plt.show()


In [ ]:
# 8) Salvar modelo, scaler e resultados de teste
model_path = '/mnt/data/diabetes_model_rf.pkl'
scaler_path = '/mnt/data/diabetes_scaler.pkl'
predictions_path = '/mnt/data/predictions_test.csv'

with open(model_path, 'wb') as f:
    pickle.dump(rf, f)

with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

# Salvar DataFrame com previsões de teste
df_test = X_test.copy()
df_test['y_true'] = y_test.values
df_test['y_pred'] = y_pred_rf
df_test['y_proba'] = y_proba_rf
df_test.to_csv(predictions_path, index=False)

print('Modelos e resultados salvos em:')
print('-', model_path)
print('-', scaler_path)
print('-', predictions_path)


---


**Observações finais:**

- O próximo passo será construir o dashboard no Power BI utilizando `predictions_test.csv` e, se necessário, incorporar o script Python para geração das previsões diretamente no Power BI. 

- Recomendações: executar validação cruzada, ajuste de hiperparâmetros com GridSearchCV, e priorizar o ajuste de recall caso o custo de falsos negativos seja alto no contexto clínico.

